In [1]:
import keypress_recognition.models as models
from keypress_recognition import dataset_p as dataset
import numpy as np
import matplotlib.pyplot as plt
import cv2
%load_ext autoreload
%autoreload 2

In [2]:
concat = False
delta = 3 if concat else 1
channels = 3 * delta

In [3]:
mod = {}
mod['black'] = models.get_black_key_model(channels)
mod['black'].load_model('keypress_recognition/models/best_black_model.tar')
mod['white'] = models.get_white_key_model(channels)
mod['white'].load_model('keypress_recognition/models/best_white_model.tar') 

Trying to load model from keypress_recognition/models/best_black_model.tar ...
done
Trying to load model from keypress_recognition/models/best_white_model.tar ...
done


In [4]:
dataset.load_all_data(color=['black', 'white'], size=['single'], keypress=True, concatenate=concat, delta=delta)

Start seperating keyboard ...
  White single width: 21px
  Black single width: 20px
  White bundle width: 37px
  Black bundle width: 36px


IntProgress(value=0, max=14273)

In trainset: 
  # of pressed black key: 8755
  # of unpressed black key: 43822
  # of pressed white key: 19083
  # of unpressed white key: 57344


IntProgress(value=0, max=5000)

In testset: 
  # of pressed black key: 761
  # of unpressed black key: 22609
  # of pressed white key: 10831
  # of unpressed white key: 23167


IntProgress(value=0, max=2993)

In valset: 
  # of pressed black key: 817
  # of unpressed black key: 10821
  # of pressed white key: 5090
  # of unpressed white key: 12125


In [5]:
def visualize_keys(note):
    """
    note: bool
    return: str
    """
    return ("Y" if note else "N")

def display_concat_img(img, idx=None):
    if concat:
        if idx is None:
            return img[:, :, :, 3:6]
        else:
            return img[idx, :, :, 3:6]
    if idx is None:
        return img
    else:
        return img[idx, :, :, :]


In [10]:
for color in ('black', 'white'):
    results = mod[color].test(dataset, size='single', color=color, concatenate=concat)
    for k, v in results.items():
        print("{:10}: {:.4f}".format(k.title(), v))

IntProgress(value=0, max=44980)

Precision : 0.9820
Recall    : 0.9921
Accuracy  : 0.9870
F1 Score  : 0.9870


IntProgress(value=0, max=45996)

Precision : 0.9263
Recall    : 0.9495
Accuracy  : 0.9370
F1 Score  : 0.9378


In [6]:
pred = {}
for color in ('black', 'white'):
    inputs = np.transpose(dataset.X['single'][color]['test'], (0, 3, 1, 2))
    print(inputs.shape)
    pred[color] = mod[color].evaluate(inputs)
    pred[color] = pred[color].numpy()
    np.save(f'{color}_prediction.npy', pred[color])

(23370, 3, 106, 20)
(33998, 3, 106, 21)


RuntimeError: CUDA out of memory. Tried to allocate 4.51 GiB (GPU 0; 7.44 GiB total capacity; 882.53 MiB already allocated; 4.08 GiB free; 2.11 GiB cached)